In [ ]:
# библиотека запросов, html-парсер, вспомогательные библиотеки для работы с PDF
!pip install requests beautifulsoup4 pymupdf pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.5 MB/s eta 0:00:00


In [ ]:
# импорты библиотек, необходимых для обработки документов
import requests
from bs4 import BeautifulSoup
import os
import fitz  # PyMuPDF
import zipfile
from tqdm.notebook import tqdm

**Определим вспомогательные функции для обработки PDF.**

Для ускорения обработки, удалим страницы, которые содержат только графическую информацию.

In [ ]:
# Функция для удаления указанных страниц
def filter_pages(doc, pages_to_remove):
    """Удаляет указанные страницы из документа."""
    pages_to_keep = [i for i in range(len(doc)) if (i + 1) not in pages_to_remove]
    new_doc = fitz.open()
    for page_num in pages_to_keep:
        new_doc.insert_pdf(doc, from_page=page_num, to_page=page_num)
    return new_doc

Архивы нужны были для создания пошаговой резервной копии.
Для удобства работы архивы были закомменчены.

In [ ]:
# Функция для скачивания, фильтрации и архивирования PDF
def split_pdf_by_pages_and_zip(pdf_url, output_folder, zip_name, pages_to_remove):
    # Скачиваем PDF
    response = requests.get(pdf_url)
    pdf_filename = os.path.join(output_folder, "temp.pdf")
    with open(pdf_filename, "wb") as f:
        f.write(response.content)

    # Открываем PDF
    doc = fitz.open(pdf_filename)

    # Удаляем указанные страницы
    filtered_doc = filter_pages(doc, pages_to_remove)

    # Создаем ZIP-архив
#    zip_path = os.path.join(output_folder, f"{zip_name}.zip")
#    with zipfile.ZipFile(zip_path, 'w') as zipf:
    for page_num in range(len(filtered_doc)):
        # Создаем новый PDF для каждой страницы
        single_page_pdf = fitz.open()
        single_page_pdf.insert_pdf(filtered_doc, from_page=page_num, to_page=page_num)
        page_filename = os.path.join(output_folder, f"page_{page_num + 1}.pdf")
        single_page_pdf.save(page_filename)
        single_page_pdf.close()

        # Добавляем страницу в ZIP-архив
#        zipf.write(page_filename, arcname=f"page_{page_num + 1}.pdf")
#        os.remove(page_filename)  # Удаляем временный файл страницы

    # Удаляем временный файл PDF
    os.remove(pdf_filename)

Собираем все функции в один конвеер.

In [ ]:
# Основная функция 0
def main_convert():
    url = "https://www.energokab.ru/zavod/reklamnyie-materialyi/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ищем все ссылки на PDF
    pdf_links = soup.find_all('a', href=True)
    pdf_links = [link['href'] for link in pdf_links if link['href'].endswith('.pdf')]

    # Список страниц для удаления для каждого каталога
    pages_to_remove = [
        [1, *range(3, 12), 338, 339],  # Для первого каталога
        [*range(1, 5), 43, 49, 127, 128, *range(130, 135), 174, 180, 258, 259, *range(261, 267)],  # Для второго каталога
        [1, 36],  # Для третьего каталога
        [1, 6, 7, 14, 15, 16],  # Для четвертого каталога
        [1, 6, 7, 14, 15, 16],  # Для пятого каталога
        [1, 6, 7, 14, 15, 16],  # Для шестого каталога
        [2, 8, 11, 25, 39]  # Для седьмого каталога
    ]

    # Обрабатываем каждый PDF
    for i, pdf_link in enumerate(pdf_links):
        if i >= len(pages_to_remove):  # Если список страниц для удаления закончился
            break

        full_pdf_url = requests.compat.urljoin(url, pdf_link)
        output_folder = f"document_{i+1}"
        os.makedirs(output_folder, exist_ok=True)

        # Разбиваем PDF, удаляем страницы и архивируем
        zip_name = f"document_{i+1}_pages"
        split_pdf_by_pages_and_zip(full_pdf_url, output_folder, zip_name, pages_to_remove[i])
        print(f"PDF {i+1} обработан. Удалены страницы: {pages_to_remove[i]}. Результат сохранен в архиве {zip_name}.zip.")

#if __name__ == "__main__":
main_convert()

PDF 1 обработан. Удалены страницы: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 338, 339]. Результат сохранен в архиве document_1_pages.zip.
PDF 2 обработан. Удалены страницы: [1, 2, 3, 4, 43, 49, 127, 128, 130, 131, 132, 133, 134, 174, 180, 258, 259, 261, 262, 263, 264, 265, 266]. Результат сохранен в архиве document_2_pages.zip.
PDF 3 обработан. Удалены страницы: [1, 36]. Результат сохранен в архиве document_3_pages.zip.
PDF 4 обработан. Удалены страницы: [1, 6, 7, 14, 15, 16]. Результат сохранен в архиве document_4_pages.zip.
PDF 5 обработан. Удалены страницы: [1, 6, 7, 14, 15, 16]. Результат сохранен в архиве document_5_pages.zip.
PDF 6 обработан. Удалены страницы: [1, 6, 7, 14, 15, 16]. Результат сохранен в архиве document_6_pages.zip.
PDF 7 обработан. Удалены страницы: [2, 8, 11, 25, 39]. Результат сохранен в архиве document_7_pages.zip.


Для перевода PDF в HTML файлы был использован установленный на удаленном сервере сервис DEDOC.
https://www.ispras.ru/technologies/dedoc/

In [ ]:
# Настройки для запроса через Dedoc
FORM_DATA = {
    "pdf_with_text_layer": "auto_tabby",
    "document_type": "other",
    "language": "rus+eng",
    "need_pdf_table_analysis": "true",
    "need_header_footer_analysis": "false",
    "is_one_column_document": "auto",
    "return_format": "html",
}

In [ ]:
# Функция отправки запроса к Dedoc
def make_request(files, form_data, upload_url):
    try:
        r = requests.post(upload_url, files=files, data=form_data)
        if not r.ok:
            print("Ошибка запроса 1", r.status_code)
            return ""
        return r.content.decode("utf-8")

    except Exception as e:
        print("Ошибка запроса 2")
        return ""

In [ ]:
# Функция конвертация одного PDF документа с помощью Dedoc
def convert_to_html(pdf_name, html_name):
    with open (pdf_name, "rb") as f:
        files = dict(file=(pdf_name, f.read()))
    dedoc_url = "http://158.255.5.135:1231/upload"
    html_content = make_request(files, FORM_DATA, dedoc_url)
    with open (html_name, "wt") as f:
        f.write(html_content)

In [ ]:
# Проверка
# convert_to_html("/content/document_4/page_10.pdf", "/content/document_4/page_10.html")

'# Проверка\nconvert_to_html("/content/document_4/page_10.pdf", "/content/document_4/page_10.html")'

Соберём вместе в единый конвеер функции необходимые для конвертации PDF в HTML с использованием пакетной обработки.

In [ ]:
# Основная функция
def main_html():
    # Папки с каталогами
    catalog_folders = [f"document_{i+1}" for i in range(7)]

    for folder in catalog_folders:
        if not os.path.exists(folder):
            print(f"Папка {folder} не найдена. Пропускаем.")
            continue

        # Создаем папку для HTML-файлов
        html_folder = os.path.join(folder, "html")
        os.makedirs(html_folder, exist_ok=True)
        print(f"Обрабатываю папку: {folder}")

        # Обрабатываем каждый PDF в папке
        for pdf_file in tqdm(os.listdir(folder)):
            if not pdf_file.endswith(".pdf"):
                continue

            pdf_path = os.path.join(folder, pdf_file)
            html_file = pdf_file.replace(".pdf", ".html")
            html_path = os.path.join(html_folder, html_file)

            # Конвертируем PDF в HTML
            convert_to_html(pdf_path, html_path)
            print(f"Файл {pdf_file} конвертирован в {html_file}.")

        # Архивируем папку с HTML-файлами
        zip_path = f"{folder}_html.zip"
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for root, _, files_in_folder in os.walk(html_folder):
                for file in files_in_folder:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=os.path.relpath(file_path, html_folder))

        print(f"Папка {folder} с HTML-файлами архивирована в {zip_path}.")

main_html()

Обрабатываю папку: document_1


  0%|          | 0/330 [00:00<?, ?it/s]

Файл page_204.pdf конвертирован в page_204.html.
Файл page_128.pdf конвертирован в page_128.html.
Файл page_191.pdf конвертирован в page_191.html.
Файл page_213.pdf конвертирован в page_213.html.
Файл page_146.pdf конвертирован в page_146.html.
Файл page_159.pdf конвертирован в page_159.html.
Файл page_121.pdf конвертирован в page_121.html.
Файл page_228.pdf конвертирован в page_228.html.
Файл page_131.pdf конвертирован в page_131.html.
Файл page_40.pdf конвертирован в page_40.html.
Файл page_328.pdf конвертирован в page_328.html.
Файл page_320.pdf конвертирован в page_320.html.
Файл page_132.pdf конвертирован в page_132.html.
Файл page_151.pdf конвертирован в page_151.html.
Файл page_82.pdf конвертирован в page_82.html.
Файл page_60.pdf конвертирован в page_60.html.
Файл page_49.pdf конвертирован в page_49.html.
Файл page_197.pdf конвертирован в page_197.html.
Файл page_163.pdf конвертирован в page_163.html.
Файл page_260.pdf конвертирован в page_260.html.
Файл page_149.pdf конвертиро

  0%|          | 0/244 [00:00<?, ?it/s]

Файл page_204.pdf конвертирован в page_204.html.
Файл page_128.pdf конвертирован в page_128.html.
Файл page_191.pdf конвертирован в page_191.html.
Файл page_213.pdf конвертирован в page_213.html.
Файл page_146.pdf конвертирован в page_146.html.
Файл page_159.pdf конвертирован в page_159.html.
Файл page_121.pdf конвертирован в page_121.html.
Файл page_228.pdf конвертирован в page_228.html.
Файл page_131.pdf конвертирован в page_131.html.
Файл page_40.pdf конвертирован в page_40.html.
Файл page_132.pdf конвертирован в page_132.html.
Файл page_151.pdf конвертирован в page_151.html.
Файл page_82.pdf конвертирован в page_82.html.
Файл page_60.pdf конвертирован в page_60.html.
Файл page_49.pdf конвертирован в page_49.html.
Файл page_197.pdf конвертирован в page_197.html.
Файл page_163.pdf конвертирован в page_163.html.
Файл page_149.pdf конвертирован в page_149.html.
Файл page_193.pdf конвертирован в page_193.html.
Файл page_114.pdf конвертирован в page_114.html.
Файл page_173.pdf конвертиро

...

Файл page_9.pdf конвертирован в page_9.html.
Файл page_4.pdf конвертирован в page_4.html.
Файл page_3.pdf конвертирован в page_3.html.
Файл page_10.pdf конвертирован в page_10.html.
Файл page_6.pdf конвертирован в page_6.html.
Файл page_5.pdf конвертирован в page_5.html.
Файл page_2.pdf конвертирован в page_2.html.
Файл page_8.pdf конвертирован в page_8.html.
Файл page_1.pdf конвертирован в page_1.html.
Файл page_7.pdf конвертирован в page_7.html.
Папка document_5 с HTML-файлами архивирована в document_5_html.zip.
Обрабатываю папку: document_6


  0%|          | 0/11 [00:00<?, ?it/s]

Файл page_9.pdf конвертирован в page_9.html.
Файл page_4.pdf конвертирован в page_4.html.
Файл page_3.pdf конвертирован в page_3.html.
Файл page_10.pdf конвертирован в page_10.html.
Файл page_6.pdf конвертирован в page_6.html.
Файл page_5.pdf конвертирован в page_5.html.
Файл page_2.pdf конвертирован в page_2.html.
Файл page_8.pdf конвертирован в page_8.html.
Файл page_1.pdf конвертирован в page_1.html.
Файл page_7.pdf конвертирован в page_7.html.
Папка document_6 с HTML-файлами архивирована в document_6_html.zip.
Обрабатываю папку: document_7


  0%|          | 0/35 [00:00<?, ?it/s]

Файл page_9.pdf конвертирован в page_9.html.
Файл page_4.pdf конвертирован в page_4.html.
Файл page_3.pdf конвертирован в page_3.html.
Файл page_29.pdf конвертирован в page_29.html.
Файл page_31.pdf конвертирован в page_31.html.
Файл page_10.pdf конвертирован в page_10.html.
Файл page_33.pdf конвертирован в page_33.html.
Файл page_6.pdf конвертирован в page_6.html.
Файл page_5.pdf конвертирован в page_5.html.
Файл page_15.pdf конвертирован в page_15.html.
Файл page_24.pdf конвертирован в page_24.html.
Файл page_23.pdf конвертирован в page_23.html.
Файл page_22.pdf конвертирован в page_22.html.
Файл page_27.pdf конвертирован в page_27.html.
Файл page_20.pdf конвертирован в page_20.html.
Файл page_25.pdf конвертирован в page_25.html.
Файл page_2.pdf конвертирован в page_2.html.
Файл page_12.pdf конвертирован в page_12.html.
Файл page_28.pdf конвертирован в page_28.html.
Файл page_8.pdf конвертирован в page_8.html.
Файл page_32.pdf конвертирован в page_32.html.
Файл page_19.pdf конвертиро

С официального сайта организации были скачены необходимые материалы в формате PDF. Материалы были разбиты на отдельные страницы и сконвертированы в HTML формат.